# Configuration

In [145]:
from pyspark.sql import SparkSession
from winerror import TYPE_E_REGISTRYACCESS

# jupyter server url : http://127.0.0.1:8888/?token=feee72ecb0de9269728ba7e915647a4e8f50dd477058e9e1

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PierreMihEval") \
    .config("spark.ui.port","4040")\
    .config("spark.executor.memory","10g")\
    .config("spark.driver.memory","4g")\
    .config("spark.jars", "/opt/spark/jars/mysql-connector-j-9.0.0.jar")\
    .getOrCreate()

In [146]:
user = "thegameadministrator"
password = "thegameadministratorpassword"
driver = "com.mysql.jdbc.Driver"
schemaName = "thegame"
properties = {"user":user,
              "driver":driver,
              "password":password
              }
connectionString = "jdbc:mysql://db"

In [147]:
# readRandomTable = spark.read \
#     .jdbc(connectionString, "mysql.engine_cost", properties=properties)
# readRandomTable.show()

# Fin de la configuration
# Début du chargement et éclatement des données

In [148]:
df_db_json = spark.read.json("/home/jovyan/data/db.jsonl")
df_db_json.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [149]:
df_sales_json = spark.read.json("/home/jovyan/data/sales.jsonl")
df_sales_json.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [150]:
from pyspark.sql.functions import col, explode, posexplode

df_player_messages_sentiment = df_db_json.select("id", explode(col("chat_messages")).alias("message"))
df_player_messages_sentiment = df_player_messages_sentiment.select(
    "id",
    col("message.date").alias("date"),
    col("message.sentiment").alias("sentiment")
)
df_player_messages_sentiment.show(truncate=False)

+---+--------------------+---------+
|id |date                |sentiment|
+---+--------------------+---------+
|0  |2020-10-20Z12:13:54Z|0.74     |
|1  |2022-12-08Z18:10:35Z|0.75     |
|1  |2021-02-26Z05:30:02Z|0.4      |
|1  |2024-07-18Z11:20:42Z|0.67     |
|1  |2023-03-12Z09:49:09Z|0.69     |
|2  |2024-01-26Z11:43:58Z|0.47     |
|2  |2020-05-17Z12:38:38Z|0.5      |
|2  |2020-09-17Z00:35:48Z|0.74     |
|2  |2023-02-05Z04:53:34Z|0.72     |
|2  |2021-11-23Z14:41:52Z|0.62     |
|2  |2024-01-04Z03:56:23Z|0.61     |
|2  |2023-11-13Z18:38:19Z|0.49     |
|2  |2022-01-26Z10:44:47Z|0.61     |
|2  |2021-10-30Z16:40:30Z|0.49     |
|2  |2021-06-12Z09:42:49Z|0.55     |
|2  |2024-06-28Z21:01:31Z|0.55     |
|2  |2021-06-12Z20:47:01Z|0.61     |
|2  |2023-06-01Z20:09:37Z|0.76     |
|2  |2022-04-16Z23:57:32Z|0.68     |
|2  |2023-08-03Z09:15:04Z|0.7      |
+---+--------------------+---------+
only showing top 20 rows



In [151]:
df_player_matches = df_db_json.select("id", explode("last_matches").alias("matches"))
df_player_matches = df_player_matches.select(col("id").alias("player_id"),
                                             col("matches.date").alias("date"),
                                             col("matches.kills").alias("kills"),
                                             col("matches.deaths").alias("deaths"),
                                             col("matches.friendly_kills").alias("friendly_kills"),
                                             col("matches.time_in_movement_ration").alias("time_in_movement_ration"),
                                             col("matches.shots_fired").alias("shots_fired"),
                                             col("matches.flags_captured").alias("flags_captured"),
                                             )
df_player_matches.show(truncate=False)

+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|player_id|date                |kills|deaths|friendly_kills|time_in_movement_ration|shots_fired|flags_captured|
+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|0        |2021-11-24Z14:37:09Z|0    |0     |1             |0.24                   |231        |0             |
|0        |2020-12-28Z19:16:48Z|1    |5     |0             |0.27                   |164        |1             |
|0        |2021-02-18Z06:19:05Z|2    |5     |1             |0.16                   |238        |2             |
|0        |2021-08-28Z10:56:50Z|3    |4     |1             |0.06                   |52         |2             |
|0        |2023-09-27Z18:11:34Z|0    |0     |0             |0.3                    |216        |0             |
|0        |2023-12-31Z06:10:56Z|6    |5     |0             |0.27                   |68         |0       

In [152]:
df_players = df_db_json.select("id", "level", "total_hours_played")
df_players.show()

+---+-----+------------------+
| id|level|total_hours_played|
+---+-----+------------------+
|  0|    3|                12|
|  1|   11|               440|
|  2|   16|               224|
|  3|    6|               240|
|  4|    8|                32|
|  5|   11|               132|
|  6|   11|                44|
|  7|   20|               880|
|  8|    3|                63|
|  9|   18|               666|
| 10|    1|                34|
| 11|   12|                60|
| 12|   18|               900|
| 13|   14|                42|
| 14|   19|               741|
| 15|   19|               437|
| 16|   14|               700|
| 17|    5|                50|
| 18|    4|                28|
| 19|   10|               490|
+---+-----+------------------+
only showing top 20 rows



In [153]:
df_sales_json.show()

+--------------------+----+
|        transactions|user|
+--------------------+----+
|[{2021-03-11, [{A...|   0|
|[{2022-01-26, [{A...|   1|
|[{2021-05-24, [{A...|   2|
|[{2024-05-27, [{A...|   3|
|[{2023-01-15, [{A...|   4|
|[{2022-05-02, [{U...|   5|
|[{2022-02-24, [{A...|   6|
|[{2022-11-15, [{A...|   7|
|[{2022-03-21, [{I...|   8|
|[{2024-03-29, [{A...|   9|
|[{2024-06-20, [{U...|  10|
|[{2020-05-06, [{U...|  11|
|[{2022-05-07, [{U...|  12|
|[{2022-01-27, [{D...|  13|
|[{2023-05-05, [{A...|  14|
|[{2024-06-27, [{G...|  15|
|[{2020-08-17, [{E...|  16|
|[{2020-09-06, [{A...|  17|
|[{2020-03-16, [{A...|  18|
|[{2023-01-11, [{A...|  19|
+--------------------+----+
only showing top 20 rows



In [154]:
from pyspark.sql.functions import posexplode
df_sales_exploded = df_sales_json.select(posexplode("transactions").alias("transaction_number", "transaction"), "user")
df_sales_exploded.show(truncate=False)

+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|transaction_number|transaction                   

In [155]:
df_sales = df_sales_exploded.select(
    col("user").alias("user_id"),
    col("transaction_number").alias("transaction_number"),
    col("transaction.date").alias("date"),
    col("transaction.products").alias("products"),
)
df_sales.show(5,truncate=False)

+-------+------------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|transaction_number|date      |products   

In [156]:
df_transaction_products_exploded = df_sales.select(
    col("user_id"),
    col("transaction_number"),
    explode(col("products")).alias("product")
)
df_transaction_products_exploded.show()

+-------+------------------+--------------------+
|user_id|transaction_number|             product|
+-------+------------------+--------------------+
|      0|                 0|{A fun air guitar...|
|      0|                 0|{A skin that give...|
|      0|                 0|{A rare dye that ...|
|      0|                 0|{Unlocks a new ba...|
|      0|                 0|{Unlocks a new ba...|
|      0|                 0|{Allows your guil...|
|      0|                 0|{A loyal pet that...|
|      0|                 0|{A pack containin...|
|      0|                 0|{Enhance your cha...|
|      0|                 1|{Unlocks a specia...|
|      0|                 1|{A bundle of 10 m...|
|      0|                 1|{A loyal pet that...|
|      0|                 1|{A powerful armor...|
|      0|                 1|{A bundle contain...|
|      0|                 1|{Grants access to...|
|      0|                 2|{A majestic outfi...|
|      0|                 2|{Allows you to ch...|


In [157]:
df_transaction_products = df_transaction_products_exploded.select(
    col("user_id"),
    col("transaction_number"),
    col("product.name").alias("name"),
    col("product.description").alias("description"),
    col("product.discount").alias("discount"),
)
df_transaction_products.show(truncate=False)

+-------+------------------+----------------------------------+----------------------------------------------------------------------------------+--------+
|user_id|transaction_number|name                              |description                                                                       |discount|
+-------+------------------+----------------------------------+----------------------------------------------------------------------------------+--------+
|0      |0                 |Special Emote: Air Guitar         |A fun air guitar emote to use during downtime.                                    |NULL    |
|0      |0                 |Rare Weapon Skin: Frostbite       |A skin that gives your weapon an icy appearance.                                  |NULL    |
|0      |0                 |Exclusive Armor Dye: Midnight Blue|A rare dye that colors your armor a deep midnight blue.                           |NULL    |
|0      |0                 |Exclusive Background Music        |U

In [158]:
df_player_transactions = df_sales.select("user_id", "transaction_number", "date")
df_player_transactions.show()

+-------+------------------+----------+
|user_id|transaction_number|      date|
+-------+------------------+----------+
|      0|                 0|2021-03-11|
|      0|                 1|2021-10-21|
|      0|                 2|2023-04-05|
|      0|                 3|2023-12-19|
|      0|                 4|2023-07-19|
|      0|                 5|2023-10-29|
|      0|                 6|2024-05-24|
|      0|                 7|2023-06-10|
|      0|                 8|2021-04-20|
|      0|                 9|2024-02-11|
|      1|                 0|2022-01-26|
|      1|                 1|2022-05-20|
|      1|                 2|2021-10-11|
|      1|                 3|2022-06-01|
|      1|                 4|2020-10-25|
|      1|                 5|2021-12-04|
|      1|                 6|2024-05-16|
|      1|                 7|2023-11-30|
|      1|                 8|2021-09-19|
|      1|                 9|2022-01-27|
+-------+------------------+----------+
only showing top 20 rows



In [159]:
df_players.show(1)
df_player_matches.show(1)
df_player_messages_sentiment.show(1)
df_player_transactions.show(1)
df_transaction_products.show(1)

+---+-----+------------------+
| id|level|total_hours_played|
+---+-----+------------------+
|  0|    3|                12|
+---+-----+------------------+
only showing top 1 row

+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|player_id|                date|kills|deaths|friendly_kills|time_in_movement_ration|shots_fired|flags_captured|
+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|        0|2021-11-24Z14:37:09Z|    0|     0|             1|                   0.24|        231|             0|
+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
only showing top 1 row

+---+--------------------+---------+
| id|                date|sentiment|
+---+--------------------+---------+
|  0|2020-10-20Z12:13:54Z|     0.74|
+---+--------------------+---------+
only showing top 1 row

+-------+------------------+

# Fin de l'éclatement des données
# Début de l'analyse et nettoyage
#### À faire côté surveillance des données dans un conteneur séparé ?

In [178]:
df_players_plus = df_players.withColumn("hours_per_level", col("total_hours_played") / col("level"))
df_players_plus.select("level", "total_hours_played", "hours_per_level").describe().show()
level_to_check = 5
df_players_plus.where(col("level") > level_to_check).orderBy("total_hours_played", ascending=True).show()

+-------+-----------------+------------------+------------------+
|summary|            level|total_hours_played|   hours_per_level|
+-------+-----------------+------------------+------------------+
|  count|             5000|              5000|              5000|
|   mean|          10.4732|           278.978|           26.7328|
| stddev|5.754954879156936|223.97090460419597|13.726860147094804|
|    min|                1|                 3|               3.0|
|    max|               20|              1000|              50.0|
+-------+-----------------+------------------+------------------+

+----+-----+------------------+---------------+
|  id|level|total_hours_played|hours_per_level|
+----+-----+------------------+---------------+
|3053|    6|                18|            3.0|
|  61|    6|                18|            3.0|
|4064|    6|                18|            3.0|
|1621|    7|                21|            3.0|
| 580|    7|                21|            3.0|
|1653|    7|         

# Fin de l'analyse et nettoyage
# Début du chargement en base de données

In [162]:
df_players.write\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", connectionString + "/" + schemaName)\
    .option("dbtable", "players")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()
df_player_matches.write\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", connectionString + "/" + schemaName)\
    .option("dbtable", "player_matches")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()
df_player_messages_sentiment.write\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", connectionString + "/" + schemaName)\
    .option("dbtable", "player_messages")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()
df_player_transactions.write\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", connectionString + "/" + schemaName)\
    .option("dbtable", "player_transactions")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()
df_transaction_products.write\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", connectionString + "/" + schemaName)\
    .option("dbtable", "transaction_products")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()